In [1]:
import os
import shlex
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vaex as vx
%matplotlib inline

In [2]:
# It doesn't properly work, because gets zer-filled tables
def fits2h5():
    for root, dirnm, filenm in os.walk(os.getcwd()):
        for filename in filenm:
            if ("Y3A2_COADD_OBJECT_SUMMARY_COORDS_" in filename):
                fits_f = os.path.join(root, filename)
                h5_f = fits_f.replace(".fits", ".hdf5")
                print(h5_f)
                cmd = "vaex convert file {0} {1}".format(fits_f, h5_f)
                cmd = shlex.split(cmd)
                sp = subprocess.Popen(cmd, stdout=subprocess.PIPE)
                outp = sp.communicate()
                sp.wait()
                print(outp)


In [5]:
# Read all tables and plot them separately (1st) and together (when the method is ok)
fn = "Y3A2_COADD_OBJECT_SUMMARY_COORDS_000015.fits"
ds = vx.open(fn)
ds.plot_widget("L", "B")#, vmin=1, background_color="white", colormap="bone")


A Jupyter Widget

In [4]:
# Read with astropy tables
from astropy.io import fits
from astropy.table import Table, Column
hdu = fits.open("Y3A2_COADD_OBJECT_SUMMARY_COORDS_000015.fits")
tab = hdu[1].data
tab.names
tnew = Table()

In [ ]:
# Transform
from astropy import units as u
from astropy.coordinates import SkyCoord
c_icrs = SkyCoord(ra=tab["ALPHAWIN_J2000"]*u.degree, dec=tab["DELTAWIN_J2000"]*u.degree, frame='icrs').galactic
print("Transformation complete")

In [ ]:
arr_l, arr_b = np.array(c_icrs.l), np.array(c_icrs.b)
tnew["L"] = tab["L"]
tnew["B"] = tab["B"]
hdu.close()
tnew["L2"] = Column(arr_l, unit="degrees")
tnew["B2"] = Column(arr_b, unit="degrees")

In [ ]:
len(tnew["B"])

In [ ]:
# Import to vaex
ds = vx.from_astropy_table(tnew)

In [ ]:
if False:
    ds.plot([["L2", "L"], ["B2", "B"]], 
            vmin=1,
            aspect="equal",
            colormap="Spectral_r", #bone_r",
            background_color='white', #visual=dict(row="what", column="subspace"),
            figsize=(12,5),
            xlabel=[r"l from ($\alpha_{J2000}$, $\delta_{J2000}$)", r"b from ($\alpha_{J2000}$, $\delta_{J2000}$)"],
            ylabel=["l", "b"],
            title=r"($\alpha_{J2000}$, $\delta_{J2000}$) to ($l$, $b$). Subsample of ~2e7 objects")
    plt.savefig("lb_comparison.png", format="png", dpi=500)

    ds.plot1d("B-B2", figsize=(8,5), 
              xlabel=r"$b$ - $b_{(\alpha, \delta)}$", 
              ylabel="N",
              label="~2e7 objects",
              color="red", lw=3)
    plt.savefig("b_hist.png", format="png", dpi=500)

    ds.plot1d("L-L2", limits=[-0.0001,0.0001],
              figsize=(8,5), 
              xlabel=r"$l$ - $l_{(\alpha, \delta)}$", 
              ylabel="N",
              color="royalblue", lw=3)
    plt.savefig("l_hist.png", format="png", dpi=500)

In [ ]:
ds.plot_widget("L", "B")
plot("enjoy")

In [ ]:
plt.show()
